In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# 设置 LLM 模型（如 llama3.1）
Settings.llm = Ollama(model="qwen2.5:7b", request_timeout=360.0)

# 设置嵌入模型（如 nomic-embed-text）
Settings.embed_model = OllamaEmbedding(model_name="bge-m3")

## 上述每一次都会加载文档并构建
## 可以从数据库中加载
# Save the index


# Later, load the index
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(
    storage_context,
    # we can optionally override the embed_model here
    # it's important to use the same embed_model as the one used to build the index
    # embed_model=Settings.embed_model,
)
query_engine = index.as_query_engine(
    # we can optionally override the llm here
    # llm=Settings.llm,
)

Loading llama_index.core.storage.kvstore.simple_kvstore from storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from storage\index_store.json.


## 查看一些预设的提示词

In [10]:
import pprint

prompts = query_engine.get_prompts()
print(prompts.keys())

pprint.pprint(prompts["response_synthesizer:text_qa_template"].get_template())
print("---"*20)
pprint.pprint(prompts["response_synthesizer:refine_template"].get_template())

dict_keys(['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template'])
('Context information is below.\n'
 '---------------------\n'
 '{context_str}\n'
 '---------------------\n'
 'Given the context information and not prior knowledge, answer the query.\n'
 'Query: {query_str}\n'
 'Answer: ')
------------------------------------------------------------
('The original query is as follows: {query_str}\n'
 'We have provided an existing answer: {existing_answer}\n'
 'We have the opportunity to refine the existing answer (only if needed) with '
 'some more context below.\n'
 '------------\n'
 '{context_msg}\n'
 '------------\n'
 'Given the new context, refine the original answer to better answer the '
 "query. If the context isn't useful, return the original answer.\n"
 'Refined Answer: ')


In [13]:
## 对预设的提示词进行更改

from llama_index.core.prompts import PromptTemplate 
my_qa_prompt_tmpl_str = ( 
 "以下是上下文信息。\n" 
 "---------------------\n" 
 "{context_str}\n" 
 "---------------------\n" 
 "根据上下文信息回答问题，不要依赖预置知识，不要编造。\n" 
 "问题: {query_str}\n" 
 "回答: " 
) 
query_engine.update_prompts( 
    {"response_synthesizer:text_qa_template":  PromptTemplate(my_qa_prompt_tmpl_str) } 
)

my_refine_tmpl_str = """
原始问题如下：{query_str}

当前已有回答：{existing_answer}

现提供以下补充上下文（仅在需要时用于完善原答案）：
------------
{context_msg}
------------

请根据新上下文对原答案进行优化，使其更准确回答该问题。若上下文无帮助，则保留原答案。

优化后的回答：
"""
query_engine.update_prompts(
    {"response_synthesizer:refine_template": PromptTemplate(my_refine_tmpl_str)}
)
prompts = query_engine.get_prompts()
pprint.pprint(prompts["response_synthesizer:text_qa_template"].get_template())
print("---"*20)
pprint.pprint(prompts["response_synthesizer:refine_template"].get_template())

('以下是上下文信息。\n'
 '---------------------\n'
 '{context_str}\n'
 '---------------------\n'
 '根据上下文信息回答问题，不要依赖预置知识，不要编造。\n'
 '问题: {query_str}\n'
 '回答: ')
------------------------------------------------------------
('\n'
 '原始问题如下：{query_str}\n'
 '\n'
 '当前已有回答：{existing_answer}\n'
 '\n'
 '现提供以下补充上下文（仅在需要时用于完善原答案）：\n'
 '------------\n'
 '{context_msg}\n'
 '------------\n'
 '\n'
 '请根据新上下文对原答案进行优化，使其更准确回答该问题。若上下文无帮助，则保留原答案。\n'
 '\n'
 '优化后的回答：\n')


## 对话历史信息

In [ ]:
from llama_index.core.prompts import PromptTemplate, ChatPromptTemplate

from llama_index.core.llms import ChatMessage

user_message = ChatMessage(('user', '如何构建Agent系统？'))